<a href="https://colab.research.google.com/github/YNNJN/colorPOOL_Data/blob/master/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 색공간 내에서의 거리 계산
- 컬러 사이의 유사도는 색공간 내 거리와 반비례 관계
    - **Palette Similarity**
    > Inverse of distance between two palettes with the closest first matched color pairs (KAIST Color Sommelier)
- 과정
    - 테마를 구성하는 5가지 컬러 중 기준 컬러(행)와의 색차가 가장 작은(유사도가 큰) 값을 따짐

In [ ]:
# connect to google drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
theme_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantizeTheme.csv')
color_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')
referenceMatrix_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/referenceMatrix.csv')

In [ ]:
# inverse distance

def inverse_distance(a,b) :
  if a[0] == b[0] and a[1] == b[1] and a[2] == b[2]: # 동일한 색인 경우
    return 1.0 
  return 1/(sum((a-b)**2))**(1/2)

In [ ]:
# 컬러와 테마간의 유사도 측정
# 테마를 구성하는 5가지 컬러 중 기준 컬러(행)와 색차가 가장 작은 값을 저장함

max = 0
columns = referenceMatrix_df.columns[1:]
matrix = pd.DataFrame(columns=columns, dtype=float)

# print(len(color_df))

for colorId in range(len(color_df)):
    rowArr = []
    for themeId in range(len(theme_df)):
        color = [color_df.iloc[colorId, 1], color_df.iloc[colorId, 2], color_df.iloc[colorId, 3]]
        max_inverse_dist = 0
        for k in range(5):
            theme = [theme_df.iloc[themeId, 3*k+1], theme_df.iloc[themeId, 3*k+2], theme_df.iloc[themeId, 3*k+3]]
            if k == 0: 
                max_inverse_dist = inverse_distance(np.array(color), np.array(theme))
            else: 
                inverse_dist = inverse_distance(np.array(color), np.array(theme))
                if max_inverse_dist < inverse_dist : 
                    max_inverse_dist = inverse_dist
        if max_inverse_dist != 1 and max < max_inverse_dist :
            max = max_inverse_dist
        rowArr.append(max_inverse_dist)
    # print(rowArr) 
    matrix = matrix.append(pd.Series(rowArr,index=columns), ignore_index=True)

matrix
# print(matrix)
matrix.to_csv('gdrive/My Drive/colorPOOL/kulerData/distanceMatrix.csv')